# Lab 2: Evaluating an N-Gram Language Model

In this lab, you will evaluate the quality of an n-gram language model using perplexity.

We have built several n-gram language models and provided an implementation for computing the probabilities. The implementation includes [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing), with assigns some probability to sequences that were never encountered during training.

First, review the implementation below to make sure that it makes sense to you.

In [1]:
import pickle
BOS = '<BOS>'
EOS = '<EOS>'
OOV = '<OOV>'
class NGramLM:
    def __init__(self, path, smoothing=0.01, verbose=False):
        with open(path, 'rb') as fin:
            data = pickle.load(fin)
        self.n = data['n']
        self.V = set(data['V'])
        self.model = data['model']
        self.smoothing = smoothing
        self.verbose = verbose

    def get_prob(self, context, token):
        
        # Take only the n-1 most recent context (Markov Assumption)
        context = tuple(context[-self.n+1:])
        # Add <BOS> tokens if the context is too short, i.e., it's at the start of the sequence
        while len(context) < (self.n-1):
            context = (BOS,) + context
        # Handle words that were not encountered during the training by replacing them with a special <OOV> token
        context = tuple((c if c in self.V else OOV) for c in context)
        if token not in self.V:
            token = OOV
        if context in self.model:
            # Compute the probability using a Maximum Likelihood Estimation and Laplace Smoothing
            count = self.model[context].get(token, 0)
            prob = (count + self.smoothing) / (sum(self.model[context].values()) + self.smoothing * len(self.V))
        else:
            # Simplified formula if we never encountered this context; the probability of all tokens is uniform
            prob = 1 / len(self.V)
        # Optional logging
        if self.verbose:
            print(f'{prob:.4n}', *context, '->', token)
        return prob

In [2]:
# Load pre-built n-gram languae models
model_unigram = NGramLM('arthur-conan-doyle.tok.train.n1.pkl')
model_bigram = NGramLM('arthur-conan-doyle.tok.train.n2.pkl')
model_trigram = NGramLM('arthur-conan-doyle.tok.train.n3.pkl')
model_4gram = NGramLM('arthur-conan-doyle.tok.train.n4.pkl')
model_5gram = NGramLM('arthur-conan-doyle.tok.train.n5.pkl')

Now it's time to see how well these models fit our data! We'll use Perplexity for this calculation, but it's up to you to implement it below.

Recall the formula for perplexity from the lecture:

$$
perplexity = 2^{\frac{-1}{n}\sum \log_2(P(w_i|w_{<i}))}
$$

Hint: you'll want to use the [`math.log2`](https://docs.python.org/3/library/math.html#math.log2) function

In [3]:
from typing import List, Tuple
import math

def perplexity(model: NGramLM, texts: List[Tuple[str]]) -> float:

    n_word = 0

    res = 0

    for text in texts:

        n_word += len(text)

        for i in range(len(text)):

            res += math.log2(model.get_prob(text[:i], text[i]))

    res = 2 ** ((-1 / n_word) * res)

    return res

# Example:
# perplexity(model_unigram, [('My', 'dear', 'Watson', '.'), ('Come', 'over', 'here', '!')])
perplexity(model_unigram, [('My', 'dear', 'Watson')])

7530.614613660964

In [4]:
# Tests
assert round(perplexity(model_unigram, [('My', 'dear', 'Watson')])) == 7531
assert round(perplexity(model_bigram, [('My', 'dear', 'Watson')])) == 24
assert round(perplexity(model_trigram, [('My', 'dear', 'Watson')])) == 521

In [5]:
toks_test = []

with open('arthur-conan-doyle.tok.test.txt', 'rt') as fin:
    for line in fin:
        toks_test.append(list(line.split()))

print(perplexity(model_unigram, toks_test))
print(perplexity(model_bigram, toks_test))
print(perplexity(model_trigram, toks_test))
print(perplexity(model_4gram, toks_test))
print(perplexity(model_5gram, toks_test))

14924.23177561972
259.53894955494343
1306.553535960716
4921.800243745181
8463.320537397633


In [6]:
toks_train = []
with open('arthur-conan-doyle.tok.train.txt', 'rt') as fin:
    for line in fin:
        toks_train.append(list(line.split()))

# your code here
print(perplexity(model_unigram, toks_train))
print(perplexity(model_bigram, toks_train))
print(perplexity(model_trigram, toks_train))
print(perplexity(model_4gram, toks_train))
print(perplexity(model_5gram, toks_train))

14976.720310596447
89.24191939089853
91.8592986931776
121.15467462849904
137.07146227562924


We can see that we get much lower perplexities when measuring on the training data, especially for the models with larger values of `n`. This suggests that the model is over-fitting to the training data.